# Latihan Membuat Machine Learning Pipeline

# Import Modul
Pada bagian bertujuan untuk mengimpor semua modul yang akan digunakan didalam proyek. Modul yang diimport adalah tensorflow dan tensorflow extended 

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

Mengatur konfogurasi direktori serta jalur penyimpanan TFX Pipeline seperti, memberi nama pipeline utama, menentukan folder root, menentuka path metadata disimpan, dan terakhir untuk membuat tempat model final.

In [2]:
PIPELINE_NAME = "twittersentiment-pipeline"
SCHEMA_PIPELINE_NAME = "twittersentiment-tfdv-schema"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)



Menentukan sumber/ path dataset yang akan digunakan

In [3]:
DATA_ROOT = "data"

Fungsi untuk menjalankan TFX didalam jupyter notebook dan menentukan direktori semua hasil pipeline 

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion

Pada sebuah machine learning pipeline, proses pengumpulan data akan dikerjakan oleh suatu komponen yang disebut data ingestion. Proyek ini menggunakan CsvExampleGen() karena dataset mempunyai format csv. Membagi dataset yang akan digunakan untuk proses training (“train”) dan evaluasi (“eval”). Pembagian ini dilakukan dengan mengatur parameter output_config dengan rasio 8:2.

In [5]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

Menjalankan example_gen

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation

Pada sebuah machine learning pipeline, proses untuk memeriksa kualitas dan mendeteksi perubahan suatu data disebut Data Validation. 

## summary statistic
Menghasilkan deskripsi statistik dari dataset, missing value, mean, standar deviasi, dan lain lain.

In [7]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
 
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

## data schema
Menyimpulkan skema data berdasarkan parameter statistik. Menghasilkan nama fitur, tipe data, dan kebutuhan kelengkapan data.

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Sentiment',INT,required,,-
'Text',BYTES,required,,-


## Mengidentifikasi anomali
Tahapan ini digunakan untuk membandingkan statistik aktual dengan skema yang diharapkan yang kemudian akan menandai anomali yang ada, seperti fitur hilang, missing value, dan lain lain

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Data Preprocessing
Setelah melakukan data validation, tahap berikutnya adalah melakukan data preprocessing. Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model.

stocksentiment_transform.py merupakan skrip python untuk melakukan proses transformasi. Transformasi yang dilakukan adalah mengubah semua huruf menjadi huruf kecil serta label dikonversi ke int64

In [13]:
TRANSFORM_MODULE_FILE = "twittersentiment_transform.py"

Melakukan transformasi kedalam dataset

In [14]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\b6b81d0a88ac4733b73756a431c919f3\assets


INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\b6b81d0a88ac4733b73756a431c919f3\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Modeling
Pada tahap ini, kita akan membuat dan melatih model menggunakan komponen Trainer.
stocksentiment_trainer.py meruapakan script python untuk melakukan proses tranformasi dataset termasuk membaginya ke dalam beberapa batch, membuat arsitektur model dengan memanfaatkan embeding layer, menentukan jumlah layer, jenis loss, dan terkhir menyimpan model.

In [15]:
TRAINER_MODULE_FILE = "twittersentiment_trainer.py"

In [16]:
from tfx.proto import trainer_pb2
 
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 128)               2176  

INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Trainer\model\6\Format-Serving\assets


50/50 [==============================] - 4s 48ms/step - loss: 0.6492 - binary_accuracy: 0.6338 - val_loss: 0.5558 - val_binary_accuracy: 0.7047
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.4251 - binary_accuracy: 0.8100
Epoch 2: val_binary_accuracy improved from 0.70469 to 0.87578, saving model to pipelines\twittersentiment-pipeline\Trainer\model\6\Format-Serving
INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Trainer\model\6\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\twittersentiment-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Analisis dan Validasi Model 
Apabila proses pengembangan model berjalan dengan lancar, tahap selanjutnya adalah membuat tahapan analisis dan validasi model. Langkah ini penting untuk memastikan bahwa model yang dihasilkan tidak hanya memberikan performa tinggi pada data pelatihan, tetapi juga umum (generalizable) terhadap data baru.

## Resolver
komponen Resolver dalam pipeline TFX untuk menemukan model terbaik sebelumnya (latest blessed model). langkah yang dilakukan adalah membandingkan model baru dengan model lama dan melakukan validasi apakah model baru layak digunakan di produksi

In [17]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Evaluator
 komponen Evaluator di pipeline TFX menggunakan TensorFlow Model Analysis (TFMA). Konfigurasi ini menentukan apa yang akan dievaluasi, bagaimana dievaluasinya, dan metrik apa yang akan digunakan.

In [18]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Sentiment')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [19]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Menampilkan visualisasi hasil evaluasi metrik seperti hasil precission atau recall

In [20]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Model Deployment

Pusher adalah komponen akhir dalam pipeline TFX yang bertugas untuk menyimpan (men-deploy) model ke direktori tertentu atau sistem deployment lainnya, jika dan hanya jika model tersebut sudah dinyatakan valid (blessed) oleh komponen Evaluator. Tahapan ini akan mendeploy model jika valid dan menyimpan kedalam direktori.

In [21]:
from tfx.components import Pusher 
from tfx.proto import pusher_pb2 
 
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model/twittersentiment-model'))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)